# Búsqueda de Hiperparámetros con SGDRegressor

Este notebook realiza la búsqueda de hiperparámetros para un modelo SGDRegressor (Stochastic Gradient Descent Regressor) con las siguientes características:

- Carga de datos de entrenamiento desde archivos CSV
- Búsqueda de hiperparámetros con Grid Search
- Validación cruzada para evaluación robusta
- División para validación (15% del conjunto de entrenamiento)
- Visualización de métricas de regresión para los mejores modelos
- Análisis comparativo de los top 3 modelos

**Autor:** ML Engineer  
**Fecha:** 6 de Septiembre, 2025

## 1. Importar Librerías Requeridas

Importar todas las librerías necesarias para el análisis, modelado y visualización.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import (
    GridSearchCV, 
    train_test_split, 
    cross_val_score, 
    StratifiedShuffleSplit
)
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score, 
    explained_variance_score,
    accuracy_score
)
from sklearn.preprocessing import StandardScaler
import joblib
from datetime import datetime
import os
import json

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✓ Librerías importadas exitosamente!")
print(f"Pandas versión: {pd.__version__}")
print(f"NumPy versión: {np.__version__}")
print(f"Scikit-learn disponible para GridSearchCV y SGDRegressor")

✓ Librerías importadas exitosamente!
Pandas versión: 2.3.2
NumPy versión: 2.3.2
Scikit-learn disponible para GridSearchCV y SGDRegressor


## 2. Cargar Datos de Entrenamiento

Cargar las características y variables objetivo desde los archivos CSV generados en el proceso de preparación de datos.

In [2]:
# Definir rutas de los archivos
X_train_path = '../data/train/X_train.csv'
y_train_path = '../data/train/y_train.csv'

print("CARGA DE DATOS DE ENTRENAMIENTO")
print("=" * 50)

# Cargar características (X_train)
try:
    X_train_full = pd.read_csv(X_train_path)
    print(f"✓ Características cargadas desde: {X_train_path}")
    print(f"  • Forma: {X_train_full.shape}")
    print(f"  • Características: {X_train_full.shape[1]}")
except FileNotFoundError:
    raise FileNotFoundError(f"Archivo no encontrado: {X_train_path}")

# Cargar variable objetivo (y_train)
try:
    y_train_full = pd.read_csv(y_train_path)
    # Si es un DataFrame con una columna, convertir a Series
    if isinstance(y_train_full, pd.DataFrame):
        y_train_full = y_train_full.iloc[:, 0]
    print(f"✓ Variable objetivo cargada desde: {y_train_path}")
    print(f"  • Forma: {y_train_full.shape}")
except FileNotFoundError:
    raise FileNotFoundError(f"Archivo no encontrado: {y_train_path}")

print(f"\n✓ Datos cargados exitosamente!")
print(f"Total de registros: {len(X_train_full):,}")

CARGA DE DATOS DE ENTRENAMIENTO
✓ Características cargadas desde: ../data/train/X_train.csv
  • Forma: (91199, 18)
  • Características: 18
✓ Variable objetivo cargada desde: ../data/train/y_train.csv
  • Forma: (91199,)

✓ Datos cargados exitosamente!
Total de registros: 91,199


## 3. División para Validación

Crear una división del 15% del conjunto de entrenamiento para validación final.

In [3]:
# División para validación (15% del dataset)
print("DIVISIÓN PARA VALIDACIÓN")
print("=" * 50)

validation_size = 0.15
random_state = 42

# Realizar la división
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, 
    y_train_full, 
    test_size=validation_size, 
    random_state=random_state,
    shuffle=True
)

print(f"División completada:")
print(f"  • Conjunto de entrenamiento: {len(X_train):,} registros ({(1-validation_size)*100:.0f}%)")
print(f"  • Conjunto de validación: {len(X_val):,} registros ({validation_size*100:.0f}%)")

# Verificar las estadísticas en ambos conjuntos
print(f"\nEstadísticas del conjunto de entrenamiento:")
print(f"  • Media: {y_train.mean():.4f}")
print(f"  • Desviación estándar: {y_train.std():.4f}")
print(f"  • Rango: [{y_train.min()}, {y_train.max()}]")

print(f"\nEstadísticas del conjunto de validación:")
print(f"  • Media: {y_val.mean():.4f}")
print(f"  • Desviación estándar: {y_val.std():.4f}")
print(f"  • Rango: [{y_val.min()}, {y_val.max()}]")

print(f"\n✓ División completada exitosamente!")

DIVISIÓN PARA VALIDACIÓN
División completada:
  • Conjunto de entrenamiento: 77,519 registros (85%)
  • Conjunto de validación: 13,680 registros (15%)

Estadísticas del conjunto de entrenamiento:
  • Media: 33.2631
  • Desviación estándar: 22.3362
  • Rango: [0, 100]

Estadísticas del conjunto de validación:
  • Media: 33.4199
  • Desviación estándar: 22.2555
  • Rango: [0, 100]

✓ División completada exitosamente!


## 5. Configuración de Búsqueda de Hiperparámetros

Definir la grilla de hiperparámetros y configurar GridSearchCV para SGDRegressor.

In [4]:
# Configuración de la grilla de hiperparámetros para SGDRegressor
print("CONFIGURACIÓN DE GRILLA DE HIPERPARÁMETROS")
print("=" * 50)

# Definir la grilla de hiperparámetros
param_grid = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],  # Parámetro de regularización
    'l1_ratio': [0.15, 0.5, 0.7, 0.9],  # Solo para elasticnet
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.001, 0.01, 0.1, 1.0],  # Tasa de aprendizaje inicial
    'max_iter': [1000, 5000, 10000]
}

print(f"Grilla de hiperparámetros definida:")
total_combinations = 1
for param, values in param_grid.items():
    print(f"  • {param}: {values}")
    total_combinations *= len(values)

print(f"\nTotal de combinaciones: {total_combinations:,}")
print(f"⚠️ Nota: Algunas combinaciones pueden no ser válidas (ej: l1_ratio solo aplica para elasticnet)")

# Configurar validación cruzada
cv_folds = 5
print(f"Validación cruzada: {cv_folds} folds")
print(f"Estimación de entrenamientos: ~{total_combinations * cv_folds // 4:,} (considerando combinaciones válidas)")

CONFIGURACIÓN DE GRILLA DE HIPERPARÁMETROS
Grilla de hiperparámetros definida:
  • loss: ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
  • penalty: ['l2', 'l1', 'elasticnet']
  • alpha: [0.0001, 0.001, 0.01, 0.1, 1.0]
  • l1_ratio: [0.15, 0.5, 0.7, 0.9]
  • learning_rate: ['constant', 'optimal', 'invscaling', 'adaptive']
  • eta0: [0.001, 0.01, 0.1, 1.0]
  • max_iter: [1000, 5000, 10000]

Total de combinaciones: 11,520
⚠️ Nota: Algunas combinaciones pueden no ser válidas (ej: l1_ratio solo aplica para elasticnet)
Validación cruzada: 5 folds
Estimación de entrenamientos: ~14,400 (considerando combinaciones válidas)


In [5]:
# Configurar GridSearchCV
print("CONFIGURACIÓN DE GRIDSEARCHCV")
print("=" * 50)

# Crear el modelo base
sgd_model = SGDRegressor(random_state=42, tol=1e-3)

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=sgd_model,
    param_grid=param_grid,
    cv=cv_folds,
    scoring='neg_mean_squared_error',  # Métrica principal para optimización
    n_jobs=-1,  # Usar todos los cores disponibles
    verbose=1,  # Mostrar progreso
    return_train_score=True,
    error_score='raise'  # Lanzar error si hay problemas
)

print(f"✓ GridSearchCV configurado:")
print(f"  • Estimador: SGDRegressor")
print(f"  • Métrica de scoring: neg_mean_squared_error")
print(f"  • Folds de CV: {cv_folds}")
print(f"  • Paralelización: Todos los cores disponibles")
print(f"  • Tolerancia: 1e-3")

print(f"\n🚀 Listo para ejecutar búsqueda de hiperparámetros...")

CONFIGURACIÓN DE GRIDSEARCHCV
✓ GridSearchCV configurado:
  • Estimador: SGDRegressor
  • Métrica de scoring: neg_mean_squared_error
  • Folds de CV: 5
  • Paralelización: Todos los cores disponibles
  • Tolerancia: 1e-3

🚀 Listo para ejecutar búsqueda de hiperparámetros...


## 6. Ejecución de Búsqueda de Hiperparámetros

Ejecutar GridSearchCV para encontrar los mejores hiperparámetros.

In [6]:
# Ejecutar búsqueda de hiperparámetros
print("🔍 INICIANDO BÚSQUEDA DE HIPERPARÁMETROS")
print("=" * 50)
print(f"Inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Esto puede tomar varios minutos...\n")

# Ejecutar grid search
start_time = datetime.now()

try:
    grid_search.fit(X_train, y_train)
    end_time = datetime.now()
    duration = end_time - start_time
    
    print(f"\n✓ Búsqueda de hiperparámetros completada!")
    print(f"Tiempo total: {duration}")
    print(f"Fin: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
except Exception as e:
    print(f"\n❌ Error durante la búsqueda: {e}")
    print("Reintentando con configuración más conservadora...")
    
    # Configuración más conservadora si hay errores
    conservative_param_grid = {
        'loss': ['squared_error', 'huber'],
        'penalty': ['l2', 'l1'],
        'alpha': [0.001, 0.01, 0.1],
        'learning_rate': ['optimal', 'constant'],
        'eta0': [0.01, 0.1],
        'max_iter': [5000, 10000]
    }
    
    grid_search_conservative = GridSearchCV(
        estimator=sgd_model,
        param_grid=conservative_param_grid,
        cv=cv_folds,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1,
        return_train_score=True
    )
    
    grid_search_conservative.fit(X_train, y_train)
    grid_search = grid_search_conservative  # Usar el resultado conservador
    end_time = datetime.now()
    duration = end_time - start_time
    
    print(f"\n✓ Búsqueda completada con configuración conservadora!")
    print(f"Tiempo total: {duration}")

🔍 INICIANDO BÚSQUEDA DE HIPERPARÁMETROS
Inicio: 2025-09-06 13:06:59
Esto puede tomar varios minutos...

Fitting 5 folds for each of 11520 candidates, totalling 57600 fits


KeyboardInterrupt: 

In [ ]:
# Analizar resultados de la búsqueda
print("RESULTADOS DE LA BÚSQUEDA DE HIPERPARÁMETROS")
print("=" * 50)

# Mejores hiperparámetros
print(f"Mejores hiperparámetros encontrados:")
for param, value in grid_search.best_params_.items():
    print(f"  • {param}: {value}")

print(f"\nMejor score (CV): {-grid_search.best_score_:.6f} (MSE)")
print(f"RMSE del mejor modelo: {np.sqrt(-grid_search.best_score_):.6f}")

# Obtener el mejor modelo
best_model = grid_search.best_estimator_
print(f"\n✓ Mejor modelo obtenido y listo para evaluación")

# Información adicional sobre convergencia
print(f"\nInformación del mejor modelo:")
print(f"  • Iteraciones hasta convergencia: {best_model.n_iter_}")
print(f"  • Coeficientes activos: {np.count_nonzero(best_model.coef_)} de {len(best_model.coef_)}")

## 7. Análisis de los Top 3 Modelos

Analizar y comparar las métricas de los 3 mejores modelos encontrados.

In [ ]:
# Obtener los top 3 modelos
print("ANÁLISIS DE LOS TOP 3 MODELOS")
print("=" * 50)

# Crear DataFrame con todos los resultados
results_df = pd.DataFrame(grid_search.cv_results_)

# Ordenar por mejor score y obtener top 3
top_3_results = results_df.nlargest(3, 'mean_test_score')

print(f"Top 3 configuraciones de hiperparámetros:")
print("=" * 40)

top_3_models = []
top_3_params = []

for i, (idx, row) in enumerate(top_3_results.iterrows(), 1):
    params = row['params']
    score = row['mean_test_score']
    std = row['std_test_score']
    
    print(f"\nModelo #{i}:")
    print(f"  MSE Score: {-score:.6f} (±{std:.6f})")
    print(f"  RMSE: {np.sqrt(-score):.6f}")
    print(f"  Parámetros: {params}")
    
    # Crear y entrenar el modelo con estos parámetros
    model = SGDRegressor(random_state=42, tol=1e-3, **params)
    model.fit(X_train_scaled, y_train)
    top_3_models.append(model)
    top_3_params.append(params)
    
    # Información adicional sobre el modelo
    print(f"  Iteraciones: {model.n_iter_}")
    print(f"  Coeficientes no-cero: {np.count_nonzero(model.coef_)}")

print(f"\n✓ Top 3 modelos entrenados y listos para evaluación")

## 8. Evaluación de Métricas de Regresión

Calcular y comparar métricas de regresión para los top 3 modelos en el conjunto de validación.

In [ ]:
# Calcular métricas para los top 3 modelos
print("EVALUACIÓN DE MÉTRICAS EN CONJUNTO DE VALIDACIÓN")
print("=" * 50)

metrics_results = []

for i, model in enumerate(top_3_models, 1):
    # Predicciones en conjunto de validación
    y_pred = model.predict(X_val_scaled)
    
    # Calcular métricas de regresión
    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    explained_var = explained_variance_score(y_val, y_pred)
    rmse = np.sqrt(mse)
    
    # Para accuracy, convertir predicciones a clases binarias
    # Asumiendo que la variable objetivo es binaria (0 o 1)
    y_pred_binary = (y_pred > 0.5).astype(int)
    y_val_binary = (y_val > 0.5).astype(int)
    accuracy = accuracy_score(y_val_binary, y_pred_binary)
    
    # Guardar resultados
    metrics = {
        'Modelo': f'Modelo #{i}',
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'Explained Variance': explained_var,
        'Accuracy': accuracy,
        'Parámetros': str(top_3_params[i-1])
    }
    metrics_results.append(metrics)
    
    print(f"\nModelo #{i} - Métricas en Validación:")
    print(f"  • MSE: {mse:.6f}")
    print(f"  • RMSE: {rmse:.6f}")
    print(f"  • MAE: {mae:.6f}")
    print(f"  • R²: {r2:.6f}")
    print(f"  • Explained Variance: {explained_var:.6f}")
    print(f"  • Accuracy: {accuracy:.6f}")
    
    # Análisis adicional de predicciones
    print(f"  • Rango de predicciones: [{y_pred.min():.3f}, {y_pred.max():.3f}]")
    print(f"  • Media de predicciones: {y_pred.mean():.3f}")

# Crear DataFrame con todas las métricas
metrics_df = pd.DataFrame(metrics_results)
print(f"\n✓ Métricas calculadas para todos los modelos")

## 9. Visualización de Métricas de Regresión

Crear gráficos comparativos de las métricas de regresión para los top 3 modelos.

In [ ]:
# Crear visualizaciones de las métricas
print("CREANDO VISUALIZACIONES DE MÉTRICAS")
print("=" * 50)

# Configurar el tamaño de la figura
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Comparación de Métricas de Regresión - Top 3 Modelos SGDRegressor', 
             fontsize=16, fontweight='bold')

# Definir métricas para visualizar
metrics_to_plot = ['MSE', 'MAE', 'R²', 'Explained Variance', 'Accuracy', 'RMSE']
colors = ['skyblue', 'lightcoral', 'lightgreen']

# Crear gráfico para cada métrica
for idx, metric in enumerate(metrics_to_plot):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    # Extraer valores de la métrica
    values = [metrics_results[i][metric] for i in range(3)]
    models = [f'Modelo #{i+1}' for i in range(3)]
    
    # Crear gráfico de barras
    bars = ax.bar(models, values, color=colors, alpha=0.7, edgecolor='black')
    
    # Configurar el gráfico
    ax.set_title(f'{metric}', fontsize=12, fontweight='bold')
    ax.set_ylabel(metric)
    ax.grid(True, alpha=0.3)
    
    # Añadir valores en las barras
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{value:.4f}', ha='center', va='bottom', fontsize=10)
    
    # Ajustar límites para mejor visualización
    if metric in ['R²', 'Explained Variance', 'Accuracy']:
        ax.set_ylim(min(0, min(values) * 1.1), max(1, max(values) * 1.1))
    else:
        ax.set_ylim(0, max(values) * 1.1)

plt.tight_layout()
plt.show()

print("✓ Visualizaciones creadas exitosamente")

In [ ]:
# Crear tabla comparativa de métricas
print("TABLA COMPARATIVA DE MÉTRICAS")
print("=" * 50)

# Mostrar tabla con métricas
display_df = metrics_df[['Modelo', 'MSE', 'RMSE', 'MAE', 'R²', 'Explained Variance', 'Accuracy']].copy()

# Formatear números para mejor lectura
for col in ['MSE', 'RMSE', 'MAE', 'R²', 'Explained Variance', 'Accuracy']:
    display_df[col] = display_df[col].round(6)

print("Resumen de métricas para los top 3 modelos:")
display(display_df)

# Identificar el mejor modelo para cada métrica
print("\nMejor modelo por métrica:")
print("-" * 30)
for metric in ['MSE', 'RMSE', 'MAE', 'R²', 'Explained Variance', 'Accuracy']:
    if metric in ['MSE', 'RMSE', 'MAE']:  # Menor es mejor
        best_idx = display_df[metric].idxmin()
    else:  # Mayor es mejor
        best_idx = display_df[metric].idxmax()
    
    best_model = display_df.loc[best_idx, 'Modelo']
    best_value = display_df.loc[best_idx, metric]
    print(f"• {metric}: {best_model} ({best_value:.6f})")

## 10. Análisis de Validación Cruzada

Realizar validación cruzada adicional en el mejor modelo para verificar su robustez.

In [ ]:
# Validación cruzada del mejor modelo
print("VALIDACIÓN CRUZADA DEL MEJOR MODELO")
print("=" * 50)

best_model = top_3_models[0]  # El primer modelo ya es el mejor

# Definir métricas para validación cruzada
cv_metrics = {
    'neg_mean_squared_error': 'MSE',
    'neg_mean_absolute_error': 'MAE',
    'r2': 'R²',
    'explained_variance': 'Explained Variance'
}

cv_results = {}

print(f"Realizando validación cruzada con {cv_folds} folds...\n")

for scoring, metric_name in cv_metrics.items():
    scores = cross_val_score(best_model, X_train_scaled, y_train, 
                           cv=cv_folds, scoring=scoring)
    
    # Para métricas negativas, convertir a positivas
    if 'neg_' in scoring:
        scores = -scores
    
    cv_results[metric_name] = scores
    
    print(f"{metric_name}:")
    print(f"  • Media: {scores.mean():.6f}")
    print(f"  • Desviación estándar: {scores.std():.6f}")
    print(f"  • Rango: [{scores.min():.6f}, {scores.max():.6f}]")
    print(f"  • Coeficiente de variación: {(scores.std()/scores.mean())*100:.2f}%")
    print()

# Evaluar estabilidad del modelo
stability_score = np.mean([cv_results[m].std() for m in cv_results])
print(f"✓ Validación cruzada completada")
print(f"Puntuación de estabilidad promedio: {stability_score:.6f}")
print(f"El modelo muestra {'alta' if stability_score < 0.05 else 'moderada' if stability_score < 0.1 else 'baja'} estabilidad")

## 11. Análisis de Predicciones vs Valores Reales

Crear visualizaciones para comparar predicciones vs valores reales.

In [ ]:
# Análisis de predicciones vs valores reales
print("ANÁLISIS DE PREDICCIONES VS VALORES REALES")
print("=" * 50)

# Crear figura con subplots para los 3 modelos
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Predicciones vs Valores Reales - Top 3 Modelos SGDRegressor', 
             fontsize=16, fontweight='bold')

for i, model in enumerate(top_3_models):
    ax = axes[i]
    
    # Hacer predicciones
    y_pred = model.predict(X_val_scaled)
    
    # Crear scatter plot
    ax.scatter(y_val, y_pred, alpha=0.6, color=colors[i])
    
    # Línea perfecta (y = x)
    min_val = min(y_val.min(), y_pred.min())
    max_val = max(y_val.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Predicción perfecta')
    
    # Configurar el gráfico
    ax.set_xlabel('Valores Reales')
    ax.set_ylabel('Predicciones')
    ax.set_title(f'Modelo #{i+1}\nR² = {r2_score(y_val, y_pred):.4f}')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Añadir línea de tendencia
    z = np.polyfit(y_val, y_pred, 1)
    p = np.poly1d(z)
    ax.plot(y_val, p(y_val), "b--", alpha=0.8, label=f'Tendencia (m={z[0]:.3f})')
    ax.legend()

plt.tight_layout()
plt.show()

print("✓ Gráficos de predicciones vs valores reales creados")

In [ ]:
# Análisis de residuos para el mejor modelo
print("ANÁLISIS DE RESIDUOS DEL MEJOR MODELO")
print("=" * 50)

best_model = top_3_models[0]
y_pred_best = best_model.predict(X_val_scaled)
residuals = y_val - y_pred_best

# Crear figura con análisis de residuos
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análisis de Residuos - Mejor Modelo SGDRegressor', fontsize=16, fontweight='bold')

# 1. Residuos vs Predicciones
axes[0, 0].scatter(y_pred_best, residuals, alpha=0.6)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Predicciones')
axes[0, 0].set_ylabel('Residuos')
axes[0, 0].set_title('Residuos vs Predicciones')
axes[0, 0].grid(True, alpha=0.3)

# 2. Histograma de residuos
axes[0, 1].hist(residuals, bins=30, alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('Residuos')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Residuos')
axes[0, 1].grid(True, alpha=0.3)

# 3. Q-Q plot de residuos
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot de Residuos')
axes[1, 0].grid(True, alpha=0.3)

# 4. Residuos vs Valores Reales
axes[1, 1].scatter(y_val, residuals, alpha=0.6)
axes[1, 1].axhline(y=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Valores Reales')
axes[1, 1].set_ylabel('Residuos')
axes[1, 1].set_title('Residuos vs Valores Reales')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas de residuos
print(f"Estadísticas de residuos:")
print(f"  • Media: {residuals.mean():.6f}")
print(f"  • Desviación estándar: {residuals.std():.6f}")
print(f"  • Sesgo: {stats.skew(residuals):.6f}")
print(f"  • Curtosis: {stats.kurtosis(residuals):.6f}")
print(f"  • Test de normalidad (Shapiro-Wilk): {stats.shapiro(residuals.sample(min(5000, len(residuals))))[1]:.6f}")

## 12. Guardar Resultados y Modelos

Guardar el mejor modelo y los resultados de la búsqueda de hiperparámetros.

In [ ]:
# Guardar resultados y modelos
print("GUARDANDO RESULTADOS Y MODELOS")
print("=" * 50)

# Crear directorio para resultados
results_dir = '../models/sgdregressor_results'
os.makedirs(results_dir, exist_ok=True)

# Guardar el scaler (muy importante para SGDRegressor)
scaler_path = os.path.join(results_dir, 'feature_scaler.joblib')
joblib.dump(scaler, scaler_path)
print(f"✓ Scaler guardado: {scaler_path}")

# Guardar el mejor modelo
best_model_path = os.path.join(results_dir, 'best_sgdregressor_model.joblib')
joblib.dump(best_model, best_model_path)
print(f"✓ Mejor modelo guardado: {best_model_path}")

# Guardar todos los top 3 modelos
for i, model in enumerate(top_3_models, 1):
    model_path = os.path.join(results_dir, f'top_{i}_model.joblib')
    joblib.dump(model, model_path)
    print(f"✓ Modelo #{i} guardado: {model_path}")

# Guardar resultados de grid search
grid_results_path = os.path.join(results_dir, 'grid_search_results.joblib')
joblib.dump(grid_search, grid_results_path)
print(f"✓ Resultados de Grid Search guardados: {grid_results_path}")

# Guardar métricas en CSV
metrics_path = os.path.join(results_dir, 'model_metrics.csv')
metrics_df.to_csv(metrics_path, index=False)
print(f"✓ Métricas guardadas: {metrics_path}")

# Crear resumen del experimento
experiment_summary = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'algorithm': 'SGDRegressor',
    'original_dataset_size': len(X_train_full),
    'training_size': len(X_train),
    'validation_size': len(X_val),
    'features_count': X_train.shape[1],
    'cv_folds': cv_folds,
    'scaling_applied': 'StandardScaler',
    'best_params': grid_search.best_params_,
    'best_cv_score': float(grid_search.best_score_),
    'best_model_iterations': int(best_model.n_iter_),
    'search_duration': str(duration),
    'validation_metrics': {
        'mse': float(metrics_results[0]['MSE']),
        'rmse': float(metrics_results[0]['RMSE']),
        'mae': float(metrics_results[0]['MAE']),
        'r2': float(metrics_results[0]['R²']),
        'explained_variance': float(metrics_results[0]['Explained Variance']),
        'accuracy': float(metrics_results[0]['Accuracy'])
    }
}

summary_path = os.path.join(results_dir, 'experiment_summary.json')
with open(summary_path, 'w') as f:
    json.dump(experiment_summary, f, indent=2)
print(f"✓ Resumen del experimento guardado: {summary_path}")

## 13. Resumen Final

Resumen completo del experimento de búsqueda de hiperparámetros.

In [ ]:
# Resumen final del experimento
print("🎉 EXPERIMENTO DE BÚSQUEDA DE HIPERPARÁMETROS COMPLETADO")
print("=" * 60)

print(f"📊 ESTADÍSTICAS DEL EXPERIMENTO:")
print(f"  • Algoritmo utilizado: SGDRegressor")
print(f"  • Dataset original: {len(X_train_full):,} registros")
print(f"  • Conjunto de entrenamiento: {len(X_train):,} registros")
print(f"  • Conjunto de validación: {len(X_val):,} registros")
print(f"  • Número de características: {X_train.shape[1]}")
print(f"  • Escalado aplicado: StandardScaler")

print(f"\n🔍 BÚSQUEDA DE HIPERPARÁMETROS:")
print(f"  • Folds de validación cruzada: {cv_folds}")
print(f"  • Tiempo total de búsqueda: {duration}")
print(f"  • Mejor convergencia: {best_model.n_iter_} iteraciones")

print(f"\n🏆 MEJORES RESULTADOS:")
print(f"  • Mejores hiperparámetros: {grid_search.best_params_}")
print(f"  • Mejor CV Score (MSE): {-grid_search.best_score_:.6f}")
print(f"  • RMSE del mejor modelo: {np.sqrt(-grid_search.best_score_):.6f}")

# Mostrar las mejores métricas en validación
best_metrics = metrics_results[0]
print(f"\n📈 MÉTRICAS EN VALIDACIÓN (MEJOR MODELO):")
print(f"  • MSE: {best_metrics['MSE']:.6f}")
print(f"  • RMSE: {best_metrics['RMSE']:.6f}")
print(f"  • MAE: {best_metrics['MAE']:.6f}")
print(f"  • R²: {best_metrics['R²']:.6f}")
print(f"  • Explained Variance: {best_metrics['Explained Variance']:.6f}")
print(f"  • Accuracy: {best_metrics['Accuracy']:.6f}")

print(f"\n🔧 CARACTERÍSTICAS DEL MODELO:")
print(f"  • Coeficientes activos: {np.count_nonzero(best_model.coef_)} de {len(best_model.coef_)}")
print(f"  • Algoritmo de optimización: Stochastic Gradient Descent")
print(f"  • Función de pérdida: {best_model.loss}")
print(f"  • Regularización: {best_model.penalty}")

print(f"\n💾 ARCHIVOS GENERADOS:")
print(f"  • Scaler: {scaler_path}")
print(f"  • Mejor modelo: {best_model_path}")
print(f"  • Top 3 modelos: {results_dir}/top_*_model.joblib")
print(f"  • Resultados completos: {grid_results_path}")
print(f"  • Métricas: {metrics_path}")
print(f"  • Resumen: {summary_path}")

print(f"\n✅ Experimento completado exitosamente!")
print(f"✅ El modelo SGDRegressor está optimizado y listo para implementación!")
print(f"⚠️ Recordatorio: Usar el scaler guardado para preprocessar nuevos datos")